<a href="https://colab.research.google.com/github/PhillNg/Kalapa-s-CreditScoring-Challenge/blob/master/kalapa_s_credit_scoring_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Input Data**

In [0]:
from google.colab import drive
drive.mount("/content/drive")

In [0]:
import os
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from wordcloud import WordCloud

In [0]:
IS_LOCAL = False
if IS_LOCAL:
  PATH = "C:/Data/"
else:
  PATH = "/content/drive/My Drive/Credit_Scoring_Challenge/"
os.listdir(PATH)

In [0]:
train_df = pd.read_csv(PATH+""+"train.csv",low_memory=False, encoding='utf-8')
test_df = pd.read_csv(PATH+""+"test.csv",low_memory=False, encoding='utf-8')

# **Data Exploration**

In [0]:
train_df.head()

In [0]:
test_df.head()

In [0]:
# some settings for displaying Pandas results
pd.set_option('display.width', 2000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.precision', 4)
pd.set_option('display.max_colwidth', -1)

In [0]:
def exploring_stats(pdf_input):
    # check rows, cols
    total_records = pdf_input.shape[0]
    total_columns = pdf_input.shape[1]
    print("Total records:", total_records)
    print("Total columns:", total_columns)

    # check dtypes
    name = []
    sub_type = []
    for n, t in pdf_input.dtypes.iteritems():
        name.append(n)
        sub_type.append(t)

    # check distinct
    ls_ndist = []
    for cname in pdf_input.columns:
        ndist = pdf_input[cname].nunique()
        pct_dist = ndist * 100.0 / total_records
        ls_ndist.append("{} ({:0.2f}%)".format(ndist, pct_dist))

    # check missing
    ls_nmiss = []
    for cname in pdf_input.columns:
        nmiss = pdf_input[cname].isnull().sum()
        pct_miss = nmiss * 100.0 / total_records
        ls_nmiss.append("{} ({:0.2f}%)".format(nmiss, pct_miss))

    # check zeros
    ls_zeros = []
    for cname in pdf_input.columns:
        try:
            nzeros = (pdf_input[cname] == 0).sum()
            pct_zeros = nzeros * 100.0 / total_records
            ls_zeros.append("{} ({:0.2f}%)".format(nzeros, pct_zeros))
        except:
            ls_zeros.append("{} ({:0.2f}%)".format(0, 0))
            continue

    # check negative
    ls_neg = []
    for cname in pdf_input.columns:
        try:
            nneg = (pdf_input[cname].astype("float") < 0).sum()
            pct_neg = nneg * 100.0 / total_records
            ls_neg.append("{} ({:0.2f}%)".format(nneg, pct_neg))
        except:
            ls_neg.append("{} ({:0.2f}%)".format(0, 0))
            continue

    # prepare output
    data = {
        "name": name,
        "sub_type": sub_type,
        "n_distinct": ls_ndist,
        "n_miss": ls_nmiss,
        "n_zeros": ls_zeros,
        "n_negative": ls_neg,
    }

    # check stats
    pdf_stats = pdf_input.describe().transpose()
    ls_stats = []
    for stat in pdf_stats.columns:
        data[stat] = []
        for cname in pdf_input.columns:
            try:
                data[stat].append(pdf_stats.loc[cname, stat])
            except:
                data[stat].append(0.0)

    # take samples
    nsample = 10
    pdf_sample = pdf_input.sample(frac=.5).head(nsample).transpose()
    pdf_sample.columns = ["sample_{}".format(i) for i in range(nsample)]

    # output
    col_ordered = ["sub_type", "n_distinct", "n_miss", "n_negative", "n_zeros",
                   "25%", "50%", "75%", "count", "max", "mean", "min", "std"] + list(pdf_sample.columns)
    pdf_data = pd.DataFrame(data).set_index("name")
    pdf_data = pd.concat([pdf_data, pdf_sample], axis=1)
    pdf_data = pdf_data[col_ordered]
    
    return pdf_data

In [0]:
# %%time
# build data missing reports
train_report = exploring_stats(train_df)
test_report = exploring_stats(test_df)

In [0]:
 %%time
 # save reports
 train_report.to_csv(PATH+""+"train_report.csv",encoding='utf-8')
 test_report.to_csv(PATH+""+"test_report.csv",encoding='utf-8')

In [0]:
train_report = pd.read_csv(PATH+""+"train_report.csv",low_memory=False, encoding='utf-8')
test_report = pd.read_csv(PATH+""+"test_report.csv",low_memory=False, encoding='utf-8')

In [0]:
train_report.head(20)

# **Data Visualization**

### Visualize functions

In [0]:
# Apply lowercase to a column in Pandas dataframe
def to_lower_case(item):
    return item.map(lambda x: x if type(x)!=str else x.lower())

train_df["province"] = to_lower_case(train_df['province'])
train_df["district"] = to_lower_case(train_df['district'])
train_df["maCv"] = to_lower_case(train_df['maCv'])

In [0]:
# Making data synchronization
def replace_data(data,key,values):
  for index, value in enumerate(data):
    if key in str(value):
      data[index] = values
    elif not str(value):
      data[index] = data[index]
  return data

replace_data(train_df["maCv"],'công nhân','cn')
replace_data(train_df["maCv"],'cong nhan','cn')
replace_data(train_df["maCv"],'cn','cn')
replace_data(train_df["maCv"],'thợ','cn')
replace_data(train_df["maCv"],'nhân viên','nv')
replace_data(train_df["maCv"],'nhân viện','nv')
replace_data(train_df["maCv"],'nhan vien','nv')
replace_data(train_df["maCv"],'trưởng phòng','nv')
replace_data(train_df["maCv"],'kế toán','nv')
replace_data(train_df["maCv"],'phó phòng','nv')
replace_data(train_df["maCv"],'ngân hàng','nv')
replace_data(train_df["maCv"],'lái xe','nvnv')
replace_data(train_df["maCv"],'chuyên viên','nv')
replace_data(train_df["maCv"],'chuyen vien','nv')
replace_data(train_df["maCv"],'nv','nv')
replace_data(train_df["maCv"],'giáo viên','gv')
replace_data(train_df["maCv"],'giao vien','gv')
replace_data(train_df["maCv"],'hiệu trưởng','gv')
replace_data(train_df["maCv"],'gv','gv')
replace_data(train_df["maCv"],'cán bộ','cb')
replace_data(train_df["maCv"],'cb','cb')
replace_data(train_df["maCv"],'can bo','cb')
replace_data(train_df["maCv"],'lãnh đạo','cb')
replace_data(train_df["maCv"],'lanh dao','cb')
replace_data(train_df["maCv"],'phó giám đốc','cb')
replace_data(train_df["maCv"],'giám đốc','cb')
replace_data(train_df["maCv"],'phó','cb')
replace_data(train_df["maCv"],'bác sỹ','cb')
replace_data(train_df["maCv"],'bác sĩ','cb')
replace_data(train_df["maCv"],'y sĩ','cb')
replace_data(train_df["maCv"],'chủ tịch','cb')
replace_data(train_df["maCv"],'công an','cb')

In [0]:
encoder = preprocessing.LabelEncoder()
train_df["province"] = encoder.fit_transform(train_df["province"].fillna('missing'))
train_df["district"] = encoder.fit_transform(train_df["district"].fillna('missing'))
train_df.head()

In [0]:
# 
def grid_bar_charts(pdf, ls_cname, ncols = 3):
    """
    Vẽ nhiều bar chart cho các thuộc tính được xếp vào grid
    Cho số lượng grid column, ta sẽ fill out bar chart cho từng cell của grid
    """
    
    # tính số dòng cần cho grid
    n_cat = len(ls_cname)    
    nrows = int(math.ceil(n_cat * 1.0 / ncols))

    # khởi tạo figure gồm nrows * ncols cho grid
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(15, 3 * nrows))
    
    # dùng tuỳ chọn này để các chart được rời nhau
    fig.set_tight_layout(False)
    
    # fill out grid
    for i in range(nrows):
        for j in range(ncols):
            # xác định vị trí tên column trong danh sách dựa vào (i, j, ncols)
            idx = i * ncols + j
            
            # khi plot hết thì dừng
            if idx == n_cat:
                break
                
            # lấy tên column cần plot
            cname = ls_cname[idx]
            s00 = pdf[~pdf[cname].isna()]
            s00 = s00.groupby(cname).size()
            
            # sắp giá trị giảm dần trước khi plot
            s00.sort_values(ascending=False).plot.bar(ax=axes[i][j], rot=45)

    # plot grid
    plt.tight_layout()
    plt.show()
    
#     
def plot_wordcloud(pdf, ls_cname):
    """
    Vẽ wordcloud cho biến có nhiều giá trị categories
    """
    
    for cname in ls_cname:
        # get sequence of types
        s00 = pdf[~pdf[cname].isna()][cname]
        text = " ".join(s00.astype(str).tolist())

        # generate wordcloud
        wordcloud = WordCloud(background_color="white", width=1600, height=800).generate(text)

        # 
        fig, ax = plt.subplots(figsize=(15, 15))
        fig.set_tight_layout(False)
        
        # plot wordcloud
        ax.imshow(wordcloud, interpolation="bilinear")
        ax.axis("off")
        ax.set_title("Word cloud of {}".format(cname), fontsize=20)
        
        #
        plt.tight_layout()
        plt.show()    
        
# 
def grid_histogram(pdf, ls_cname, ncols = 3):
    """
    Vẽ nhiều histogram cho các thuộc tính được xếp vào grid
    Cho số lượng grid column, ta sẽ fill out histogram cho từng cell của grid
    """
    
    # tính số dòng cần cho grid
    n_cat = len(ls_cname)    
    nrows = int(math.ceil(n_cat * 1.0 / ncols))

    # khởi tạo figure gồm nrows * ncols cho grid
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(15, 4 * nrows))
    
    # dùng tuỳ chọn này để các chart được rời nhau
    fig.set_tight_layout(False)
    
    # fill out grid
    for i in range(nrows):
        for j in range(ncols):
            
            # xác định vị trí tên column trong danh sách dựa vào (i, j, ncols)
            idx = i * ncols + j
            
            # khi plot hết thì dừng
            if idx == n_cat:
                break
                
            cname = ls_cname[idx]
            s00 = pdf[~pdf[cname].isna()][cname]
            s00.plot(kind="hist", ax=axes[i][j], rot=45, title=cname)
            
    plt.tight_layout()
    plt.show()        
    
# 
def plot_continuous_data(s00, title):
    """
    Quan sát continuous data bằng histogram và boxplot
    """
    
    # khởi tạo figure
    fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
    
    # plot
    s00.hist(bins=30, ax=ax1)
    s00.plot.box(ax=ax2)
    
    #
    plt.suptitle(title)
    plt.show()


### Visualize for categorical data

In [0]:
# check categorical data attributes
ls_cat_name = train_report[train_report["sub_type"] == "object"]["name"].tolist()
ls_cat_name

In [0]:
# if number of category is small we could use bar chart, otherwise use cloud chart
pdf_meta00 = train_report[train_report["name"].isin(ls_cat_name)][["name", "n_distinct"]]
pdf_meta00["chart"] = pdf_meta00["n_distinct"].apply(lambda x: "wordcloud" if int(x.split()[0]) > 10 else "bar")
pdf_meta00

In [0]:
ls_cat_bar = pdf_meta00.query("chart == 'bar'")["name"].tolist()
grid_bar_charts(train_df, ls_cat_bar)

In [0]:
ls_cat_wordcloud = pdf_meta00.query("chart == 'wordcloud'")["name"].tolist()
plot_wordcloud(train_df, ls_cat_wordcloud)

### Visualization for numerical data

In [0]:
# check numerical data attributes
ls_num_name = train_report[train_report["sub_type"] == "int64"]["name"].tolist()
ls_num_name

In [0]:
# if number of distinct values is small we could use bar chart, otherwise use histogram
pdf_meta00 = train_report[train_report["name"].isin(ls_num_name)][["name", "n_distinct"]]
pdf_meta00["chart"] = pdf_meta00["n_distinct"].apply(lambda x: "histogram" if int(x.split()[0]) > 10 else "bar")
pdf_meta00

In [0]:
ls_num_bar = pdf_meta00.query("chart == 'bar'")["name"].tolist()
grid_bar_charts(train_df, ls_num_bar)

In [0]:
ls_num_hist = pdf_meta00.query("chart == 'histogram'")["name"].tolist()
grid_histogram(train_df, ls_num_hist)

In [0]:
# check continuous data attributes
ls_continuous_name = train_report[train_report["sub_type"] == "float64"]["name"].tolist()
ls_continuous_name

In [0]:
for cname in ls_continuous_name:
    s00 = train_df[~train_df[cname].isna()][cname]    
    plot_continuous_data(s00, cname)

# **Feature engineering**

**“Coming up with features is difficult, time-consuming, requires expert knowledge. “Applied machine learning” is basically feature engineering.”**
              —  Andrew Ng, Machine Learning and AI via Brain simulations

**“Feature engineering is the process of transforming raw data into features that better represent the underlying problem to the predictive models, resulting in improved model accuracy on unseen data.”**
— Dr. Jason Brownlee from machinelearningmastery.com

In [25]:
train_report.head(10)

,name,sub_type,n_distinct,n_miss,n_negative,n_zeros,25%,50%,75%,count,max,mean,min,std,sample_0,sample_1,sample_2,sample_3,sample_4,sample_5,sample_6,sample_7,sample_8,sample_9
0,id,int64,30000 (100.00%),0 (0.00%),0 (0.00%),1 (0.00%),7499.75,14999.5,22499.25,30000.0,29999.0,14999.5000,0.0,8660.3984,29276,2141,12270,11371,17739,6186,29931,21900,12825,2428
1,label,int64,2 (0.01%),0 (0.00%),0 (0.00%),29514 (98.38%),0.00,0.0,0.00,30000.0,1.0,0.0162,0.0,0.1262,0,0,0,0,0,0,0,0,0,0
2,province,object,64 (0.21%),12818 (42.73%),0 (0.00%),0 (0.00%),0.00,0.0,0.00,0.0,0.0,0.0000,0.0,0.0000,tỉnh hà tĩnh,NaN,NaN,thành phố hà nội,tỉnh phú yên,NaN,NaN,tỉnh phú thọ,tỉnh đồng nai,NaN
3,district,object,717 (2.39%),12849 (42.83%),0 (0.00%),0 (0.00%),0.00,0.0,0.00,0.0,0.0,0.0000,0.0,0.0000,huyện thạch hà,NaN,NaN,huyện mỹ đức,huyện tuy an,NaN,NaN,huyện đoan hùng,thành phố biên hòa,NaN
4,age_source1,float64,64 (0.21%),12811 (42.70%),0 (0.00%),4 (0.01%),26.00,31.0,39.00,17189.0,71.0,33.0410,0.0,9.1877,32.0,NaN,NaN,33.0,31.0,NaN,NaN,33.0,41.0,NaN
5,age_source2,float64,69 (0.23%),9678 (32.26%),1 (0.00%),0 (0.00%),26.00,31.0,38.00,20322.0,89.0,32.8720,-1.0,9.0962,32.0,41.0,NaN,33.0,31.0,NaN,NaN,33.0,41.0,32.0
6,maCv,object,561 (1.87%),9679 (32.26%),0 (0.00%),0 (0.00%),0.00,0.0,0.00,0.0,0.0,0.0000,0.0,0.0000,cb,none,NaN,cn,nv,NaN,NaN,cn,none,none
7,FIELD_1,int64,2 (0.01%),0 (0.00%),0 (0.00%),6740 (22.47%),1.00,1.0,1.00,30000.0,1.0,0.7753,0.0,0.4174,1,1,1,1,1,1,1,1,1,1
8,FIELD_2,float64,2 (0.01%),463 (1.54%),0 (0.00%),6996 (23.32%),1.00,1.0,1.00,29537.0,1.0,0.7631,0.0,0.4252,1.0,1.0,1.0,1.0,1.0,NaN,0.0,1.0,1.0,1.0
9,FIELD_3,float64,513 (1.71%),463 (1.54%),6996 (23.32%),0 (0.00%),343.00,1452.0,2913.00,29537.0,8037.0,1731.2635,-1.0,1472.5374,711.0,729.0,349.0,346.0,711.0,NaN,-1.0,2898.0,2903.0,4009.0
